# Data Preparation

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('crop_production_data.csv',low_memory=True)

In [14]:
df.head()

,State,District,Crop,Year,Season,Area,Area Units,Production,Production Units,Yield
0,Andaman and Nicobar Islands,NICOBARS,Arecanut,2001-02,Kharif,1254.0,Hectare,2061.0,Tonnes,1.643541
1,Andaman and Nicobar Islands,NICOBARS,Arecanut,2002-03,Whole Year,1258.0,Hectare,2083.0,Tonnes,1.655803
2,Andaman and Nicobar Islands,NICOBARS,Arecanut,2003-04,Whole Year,1261.0,Hectare,1525.0,Tonnes,1.209358
3,Andaman and Nicobar Islands,NORTH AND MIDDLE ANDAMAN,Arecanut,2001-02,Kharif,3100.0,Hectare,5239.0,Tonnes,1.690000
4,Andaman and Nicobar Islands,SOUTH ANDAMANS,Arecanut,2002-03,Whole Year,3105.0,Hectare,5267.0,Tonnes,1.696296


In [15]:
df_state=pd.DataFrame(data=df.State.unique()).reset_index().rename({'index':'state_id',0:'state_name'},axis=1)
df_state.head()
#df_state.to_csv('dim_state.csv',index=False)

,state_id,state_name
0,0,Andaman and Nicobar Islands
1,1,Andhra Pradesh
2,2,Arunachal Pradesh
3,3,Assam
4,4,Bihar


In [16]:
df_district = df[['District','State']]\
    .drop_duplicates()\
    .reset_index()\
    .drop('index',axis=1)\
    .reset_index()\
    .rename({'index':'district_id','District':'district_name','State':'state_name'},axis=1)
df_district.head()
# df_district.to_csv('dim_district.csv',index=False)

,district_id,district_name,state_name
0,0,NICOBARS,Andaman and Nicobar Islands
1,1,NORTH AND MIDDLE ANDAMAN,Andaman and Nicobar Islands
2,2,SOUTH ANDAMANS,Andaman and Nicobar Islands
3,3,ANANTAPUR,Andhra Pradesh
4,4,EAST GODAVARI,Andhra Pradesh


In [17]:
df_crop = df[['Crop','Production Units']]\
    .drop_duplicates()\
    .reset_index()\
    .drop('index',axis=1)\
    .reset_index()\
    .rename({'index':'crop_id','Crop':'crop_name','Production Units':'crop_measure'},axis=1)
df_crop.head()
# df_crop.to_csv('dim_crop.csv',index=False)

,crop_id,crop_name,crop_measure
0,0,Arecanut,Tonnes
1,1,Banana,Tonnes
2,2,Black pepper,Tonnes
3,3,Cashewnut,Tonnes
4,4,Coconut,Nuts


In [18]:
df_year = pd.DataFrame(data=df.Year.unique()).reset_index().rename({'index':'year_id',0:'year_name'},axis=1)
df_year.head()
# df_year.to_csv('dim_year.csv',index=False)

,year_id,year_name
0,0,2001-02
1,1,2002-03
2,2,2003-04
3,3,2004-05
4,4,2005-06


In [19]:
df_season = pd.DataFrame(data=df.Season.unique()).reset_index().rename({'index':'season_id',0:'season_type'},axis=1)
df_season.head()
# df_season.to_csv('dim_season.csv',index=False)

,season_id,season_type
0,0,Kharif
1,1,Whole Year
2,2,Rabi
3,3,Autumn
4,4,Summer


In [20]:
df_area = pd.DataFrame(data=df.Area.unique()).reset_index().rename({'index':'area_id',0:'area_value'},axis=1)
df_area.head()
# df_area.to_csv('dim_area.csv',index=False)

,area_id,area_value
0,0,1254.0
1,1,1258.0
2,2,1261.0
3,3,3100.0
4,4,3105.0


In [21]:
fact_table = df.merge(df_state,left_on='State',right_on='state_name')\
    .merge(df_district,left_on='District',right_on='district_name')\
    .merge(df_year,left_on='Year',right_on='year_name')\
    .merge(df_season,left_on='Season',right_on='season_type')\
    .merge(df_area,left_on='Area',right_on='area_value')\
    .merge(df_crop,left_on='Crop',right_on='crop_name')\
    [['district_id','state_id','year_id','season_id','crop_id','Production','Yield']]\
    .dropna(axis=0).reset_index().rename({'index':'id'},axis=1)

fact_table.head()
# fact_table.to_csv('production_fact.csv',index=False)

,id,district_id,state_id,year_id,season_id,crop_id,Production,Yield
0,0,0,0,0,0,0,2061.0,1.643541
1,1,0,0,20,0,0,2000.0,1.594896
2,2,45,3,14,2,0,986.0,0.786284
3,3,45,3,17,2,0,1253.0,0.999203
4,4,216,16,2,1,0,3.0,1.500000
